In [6]:
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding
import os

In [26]:
openai.api_key = "key"

In [27]:
#load the data with news articles
news = pd.read_csv("MIND/newsTEST.tsv", sep='\t', header=None) #document with news description
news.columns =['ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
news['Content'] = news['Title'].fillna('') + ' ' + news['Abstract'].fillna('')
news = news.drop(['URL', 'Title Entities', 'Abstract Entities', 'Title', 'Abstract'], axis=1)
news.head()

,ID,Category,SubCategory,Content
0,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O..."
1,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...
2,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...
3,N45794,news,newscrime,Four flight attendants were arrested in Miami'...
4,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...


In [28]:
#function transforms the values of column from text into embeddings
def hashable_column(column_value):
    if isinstance(column_value, dict):
        hashable_items = []
        
        for key, value in column_value.items():
            if key == 'Content':
                # Convert 'Content' value to embedding
                value = get_embedding(value, engine="text-embedding-ada-002")
            elif isinstance(value, dict):
                value = hashable_column(value)
            elif isinstance(value, list):
                print("Unhashable list value at key:", key, "with value:", value)
            elif isinstance(value, str):
                value = value.replace("\n", " ")
            hashable_items.append((key, value))
        return tuple(sorted(hashable_items))
    else:
        if isinstance(column_value, str):
            return get_embedding(column_value, engine="text-embedding-ada-002")
        else:
            return column_value

In [29]:
#save news embeddings 
news['Content_emb'] = news['Content'].apply(hashable_column)

In [30]:
news.to_csv('embeddings/news_emb_TEST.csv', index=False)

In [31]:
news = pd.read_csv("embeddings/news_emb_TEST.csv") #document with news content
news.head()

,ID,Category,SubCategory,Content,Content_emb
0,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","[-0.020967688411474228, -0.020634232088923454,..."
1,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...,"[-0.008319429121911526, 0.0013667173916473985,..."
2,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,"[-0.042908210307359695, -0.018993420526385307,..."
3,N45794,news,newscrime,Four flight attendants were arrested in Miami'...,"[-0.007930373772978783, -0.0221870094537735, 0..."
4,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...,"[-0.020153459161520004, -0.044797133654356, 0...."
